In [1]:
import os
import torch
import numpy as np

from torch.utils.data import TensorDataset, DataLoader
from os import path

from src.models import *
from src.methods import *
from src.utils import *

data_path = path.join(os.getcwd(), 'project_data/')

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [2]:
X_test = np.load(data_path+"X_test.npy")
y_test = np.load(data_path+"y_test.npy")
person_train_valid = np.load(data_path+"person_train_valid.npy")
X_train_valid = np.load(data_path+"X_train_valid.npy")
y_train_valid = np.load(data_path+"y_train_valid.npy")
person_test = np.load(data_path+"person_test.npy")


In [3]:
y_train_valid-=769
y_test-=769

In [4]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [5]:
batch_size = 64

In [6]:
X_train_valid = torch.tensor(X_train_valid, dtype=torch.float)
y_train_valid = torch.tensor(y_train_valid, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)

train_valid_dataset = TensorDataset(X_train_valid, y_train_valid)
test_dataset = TensorDataset(X_test, y_test)

num_train = int(0.8 * len(train_valid_dataset))
num_valid = len(train_valid_dataset) - num_train
train_dataset, valid_dataset = torch.utils.data.random_split(train_valid_dataset, [num_train, num_valid])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

train(model, train_loader, valid_loader, criterion, optimizer, device=device, epochs=50)

test_accuracy = evaluate(model, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

In [8]:
lstm = LSTMModel(input_size=1000)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

lstm = lstm.to(device)
train(lstm, train_loader, valid_loader, criterion, optimizer, device=device)

test_accuracy = evaluate(lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

  0%|          | 0/20 [00:00<?, ?it/s]

 50%|█████     | 10/20 [00:12<00:11,  1.14s/it]

Epoch [10/20], Loss: 0.1329, Validation Accuracy: 31.21%


100%|██████████| 20/20 [00:25<00:00,  1.27s/it]

Epoch [20/20], Loss: 0.0722, Validation Accuracy: 29.08%
Test Accuracy: 28.67%


In [7]:
cnn_lstm = CNN_LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

cnn_lstm = cnn_lstm.to(device)
train(cnn_lstm, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100)

test_accuracy = evaluate(cnn_lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 10%|█         | 10/100 [00:12<01:48,  1.21s/it]

Epoch [10/100], Loss: 1.1323, Validation Accuracy: 45.86%


 20%|██        | 20/100 [00:24<01:36,  1.21s/it]

Epoch [20/100], Loss: 0.4571, Validation Accuracy: 52.96%


 30%|███       | 30/100 [00:36<01:24,  1.20s/it]

Epoch [30/100], Loss: 0.2133, Validation Accuracy: 54.61%


 35%|███▌      | 35/100 [00:43<01:20,  1.24s/it]

Early stopping!
Test Accuracy: 54.85%
